In [37]:
import pandas as pd
import numpy as np
import re

In [14]:
weibo_m = pd.read_csv('./data/Weibo_M.csv')
weibo = pd.read_csv('./data/Weibo.csv')
wechat = pd.read_csv('./data/Wechat.csv')
news_forum = pd.read_csv('./data/News_forum.csv')

In [15]:
weibo_m = weibo_m[['COMMENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
weibo = weibo[['COMMENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
wechat = wechat[['CONTENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
news_forum = news_forum[['CONTENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')

In [16]:
weibo_m.columns = ['CONTENT','AREAS AFFECTED']
weibo.columns = ['CONTENT','AREAS AFFECTED']

In [17]:
data = pd.concat([weibo_m,weibo,wechat,news_forum])

In [19]:
data['CONTENT'] = data['CONTENT'].str.replace('\n', '')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('life', 'Life')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('sponsored', 'Sponsored')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('news', 'News')

In [20]:
data['AREAS AFFECTED'].unique()

array(['Life Comprehend', 'Products/Service', 'Health Information',
       'Ads/Marketing', 'Customer Service', 'Irrelevant Ads',
       'Charity Events', 'Survey/Questions', 'Contact Information', 'Fund',
       'Corporate Brands', 'Sponsored Events', 'Volunteering Activity',
       'Products', 'Website Issues', 'Corporate News', 'Recruitment',
       'Agent Issues', 'Employment', 'Stocks&Earnings',
       'General Mentioned', 'Employee Information', 'General Info', 'Agent'], dtype=object)

In [25]:
for x in data['AREAS AFFECTED'].unique():
    if(len(data[data['AREAS AFFECTED'] == x]['CONTENT'])>2000):
        np.savetxt('./data/'+ x.replace('/',' ') +'.utf8',\
           data[data['AREAS AFFECTED'] == x]['CONTENT'][:2000].values, fmt='%s')
    else:
        np.savetxt('./data/'+ x.replace('/',' ') +'.utf8',\
               data[data['AREAS AFFECTED'] == x]['CONTENT'].values, fmt='%s')

In [26]:
data.shape

(30894, 2)

In [33]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(ur"[^\u4e00-\u9fff]", " ", string)
    #string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    #string = re.sub(r"\'s", " \'s", string)
    #string = re.sub(r"\'ve", " \'ve", string)
    #string = re.sub(r"n\'t", " n\'t", string)
    #string = re.sub(r"\'re", " \'re", string)
    #string = re.sub(r"\'d", " \'d", string)
    #string = re.sub(r"\'ll", " \'ll", string)
    #string = re.sub(r",", " , ", string)
    #string = re.sub(r"!", " ! ", string)
    #string = re.sub(r"\(", " \( ", string)
    #string = re.sub(r"\)", " \) ", string)
    #string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    #return string.strip().lower()
    return string.strip()

In [35]:
def seperate_line(line):
    return ''.join([word + ' ' for word in line])

In [39]:
lines = list(open('./data/Corporate Brands.utf8', "r").readlines())
lines = [clean_str(seperate_line(line.decode('utf-8'))) for line in lines]

lines

[u'\u8f6c \u53d1 \u5fae \u535a \u4e2d \u5b8f \u4fdd \u9669 \u4e2d \u5b8f \u4eba \u5bff \u575a \u5b9a \u505a \u5ba2 \u6237 \u5065 \u5eb7 \u7684 \u4fdd \u62a4 \u795e \u7406 \u8d54 \u6848 \u4f8b \u5728 \u91d1 \u878d \u754c \u9886 \u822a \u4e2d \u56fd \u91d1 \u878d \u884c \u4e1a \u521b \u65b0 \u53d1 \u5c55 \u9ad8 \u5cf0 \u8bba \u575b \u66a8 \u7b2c \u4e09 \u5c4a \u91d1 \u878d \u884c \u4e1a \u5e74 \u5ea6 \u8bc4 \u9009 \u4e2d \u8363 \u83b7 \u9886 \u822a \u4e2d \u56fd \u5e74 \u5ea6 \u8bc4 \u9009 \u6700 \u4f73 \u4fdd \u9669 \u7406 \u8d54 \u5956 \u5e76 \u540c \u65f6 \u83b7 \u9009 \u4e3a \u4e2d \u56fd \u4fdd \u9669 \u62a5 \u4e2d \u56fd \u4fdd \u9669 \u4e1a \u6708 \u5ea6 \u5f71 \u54cd \u529b \u5341 \u5927 \u8d54 \u6848 \u699c \u5355 \u6708 \u699c',
 u'\u8f6c \u53d1 \u5fae \u535a \u4e2d \u5b8f \u4fdd \u9669 \u4e2d \u5b8f \u4eba \u5bff \u575a \u5b9a \u505a \u5ba2 \u6237 \u5065 \u5eb7 \u7684 \u4fdd \u62a4 \u795e \u7406 \u8d54 \u6848 \u4f8b \u5728 \u91d1 \u878d \u754c \u9886 \u822a \u4e2d \u56fd \u91d

In [40]:
x

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
